In [1]:
#@markdown **You need to sign up for [DagsHub](https://dagshub.com/user/sign_up) , then enter the name of the repository you'd like to create, and your username and email.**

#@markdown Enter the repository name for the project:
REPO_NAME= "komrag" #@param {type:"string"}

#@markdown Enter the username of your DAGsHub account:
USER_NAME = "OsipovStas" #@param {type:"string"}

#@markdown Enter the email for your DAGsHub account:
EMAIL = "stasstels@gmail.com" #@param {type:"string"}

#@markdown ---

import IPython

In [2]:
from dotenv import load_dotenv

load_dotenv("../.env", override=True)

In [3]:
import os

DH_TOKEN = os.environ['LLM_MATH_COMP_DH_TOKEN']

In [4]:
import mlflow
import dagshub
import os
#DH_TOKEN = os.environ['LLM_MATH_COMP_DH_TOKEN']
username = f'{USER_NAME}'  # Replace with your DagsHub username
repository = f'{REPO_NAME}'  # Replace with your repository name
token = f'{DH_TOKEN}'  # Replace with your DagsHub token
os.environ['MLFLOW_TRACKING_USERNAME'] = username
os.environ['MLFLOW_TRACKING_PASSWORD'] = token
dagshub.auth.add_app_token(token)
dagshub.init(repo_name=REPO_NAME, repo_owner=USER_NAME)
mlflow.set_tracking_uri(f"https://dagshub.com/{USER_NAME}/{REPO_NAME}.mlflow")